In [ ]:
#  import lib
import os
import matplotlib
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import zipfile
import requests
from sklearn import preprocessing
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [ ]:
# reading 
BATCH_SIZE = 20
NUM_CLASSES = 200
NUM_IMAGES_PER_CLASS = 500
NUM_IMAGES = NUM_CLASSES * NUM_IMAGES_PER_CLASS
TRAINING_IMAGES_DIR = './tiny-imagenet-200/train/'
TRAIN_SIZE = NUM_IMAGES

NUM_VAL_IMAGES = 10000
VAL_IMAGES_DIR = './tiny-imagenet-200/val/'

IMAGE_SIZE = 64
NUM_CHANNELS = 3
IMAGE_ARR_SIZE = IMAGE_SIZE * IMAGE_SIZE * NUM_CHANNELS
IMAGES_URL = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'

In [ ]:
! git all-go

# Reference 
1. Python3, S., Jungi, K., Shapovalov, R., Stuart, V., Coutinho, T., & ALBARÈDE, P. et al. (2013). StringIO in Python3. Stack Overflow. Retrieved 1 February 2019, from https://stackoverflow.com/questions/11914472/stringio-in-python3